In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import glob
import pickle
import numpy as np
import random
from tqdm import tqdm
from music21 import converter, instrument, note, chord, stream
from music21.stream.base import Score
import tensorflow as tf
from pathlib import Path
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, BatchNormalization, Input, concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [ ]:
def prepare_sequences(notes, n_vocab):
	sequence_length = 128

	# get total vocab of component
	total_comp = sorted(set(item for item in notes))

	 # create a dictionary to map vocab to integers
	note_index_map = dict((note, number) for number, note in enumerate(total_comp))

	network_input = []
	network_output = []

	# create input sequences and outputs
	for i in range(0, len(notes) - sequence_length, 1):
		sequence_input = notes[i:i + sequence_length]
		sequence_output = notes[i + sequence_length]
		network_input.append([note_index_map[char] for char in sequence_input])
		network_output.append(note_index_map[sequence_output])

	n_patterns = len(network_input)

	# reshape for LSTM compatibility
	network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

	# normalise input
	network_input = network_input / float(n_vocab)

	network_output = to_categorical(network_output)

	return (network_input, network_output)

In [ ]:
def create_input_branch(input_data, name, lstm_units=256, dropout_rate=0.2):

    input_layer = Input(shape=(input_data.shape[1], input_data.shape[2]), name=f"input_{name}")
    x = LSTM(
        lstm_units,
        return_sequences=True,
        name=f"lstm_{name}"
    )(input_layer)
    x = Dropout(dropout_rate, name=f"dropout_{name}")(x)

    return input_layer, x

In [ ]:
def create_output_branch(x, n_vocab, name, dense_units=128, dropout_rate=0.3):

    x = Dense(dense_units, activation='relu', name=f"dense_{name}")(x)
    x = BatchNormalization(name=f"bn_{name}")(x)
    x = Dropout(dropout_rate, name=f"dropout_{name}")(x)
    output = Dense(n_vocab, activation='softmax', name=name)(x)

    return output

In [ ]:
def create_network(network_input_notes, n_vocab_notes,
                  network_input_durations, n_vocab_durations,
                  network_input_offsets, n_vocab_offsets):

    # Create input branches
    input_notes_layer, input_notes = create_input_branch(network_input_notes, "notes")
    input_durations_layer, input_durations = create_input_branch(network_input_durations, "durations")
    input_offsets_layer, input_offsets = create_input_branch(network_input_offsets, "offsets")

    # Concatenate the three input branches
    combined = concatenate([input_notes, input_durations, input_offsets], name="combined_features")

    # Process combined features
    x = LSTM(512, return_sequences=True, name="lstm_combined_1")(combined)
    x = Dropout(0.3, name="dropout_combined_1")(x)
    x = LSTM(512, name="lstm_combined_2")(x)
    x = BatchNormalization(name="bn_combined")(x)
    x = Dropout(0.3, name="dropout_combined_2")(x)
    x = Dense(256, activation='relu', name="dense_combined")(x)

    # Create output branches
    output_notes = create_output_branch(x, n_vocab_notes, "Note")
    output_durations = create_output_branch(x, n_vocab_durations, "Duration")
    output_offsets = create_output_branch(x, n_vocab_offsets, "Offset")

    # Create and compile model
    model = Model(
        inputs=[input_notes_layer, input_durations_layer, input_offsets_layer],
        outputs=[output_notes, output_durations, output_offsets]
    )

    model.compile(loss='categorical_crossentropy', optimizer='adam')


    return model

In [ ]:
def train(model, network_input_notes, network_input_durations, network_input_offsets,
          network_output_notes, network_output_durations, network_output_offsets,
          sequence_length, dropout_rate, validation_split=0.2):
    """ Train the neural network, now with a validation set """
    filepath = Path(f"/content/drive/MyDrive/finalyearproject/gridsearch/weights/weights-seq{sequence_length}-drop{dropout_rate}-{{epoch:02d}}-{{loss:.4f}}-bigger.keras")

    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )

    early_stopping = EarlyStopping(
        monitor='loss',
        patience=10,
        restore_best_weights=True
    )
    callbacks_list = [early_stopping]

    # After preparing all network_inputs and outputs:
    indices = np.arange(len(network_input_notes))
    np.random.shuffle(indices)

    # Apply shuffle to all inputs and outputs
    network_input_notes = network_input_notes[indices]
    network_input_durations = network_input_durations[indices]
    network_input_offsets = network_input_offsets[indices]
    network_output_notes = network_output_notes[indices]
    network_output_durations = network_output_durations[indices]
    network_output_offsets = network_output_offsets[indices]

    # Include validation_split here so that validation metrics are tracked
    history = model.fit(
        [network_input_notes, network_input_durations, network_input_offsets],
        [network_output_notes, network_output_durations, network_output_offsets],
        epochs=50,
        batch_size=128,
        validation_split=validation_split,
        callbacks=callbacks_list,
        verbose=1
    )
    return history

In [ ]:
def grid_search_training():
    """ Perform grid search to find best hyperparameters with training and validation tracking """
    sequence_lengths = [64, 128, 192]
    dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5]
    results = []

    # Load notes
    notes_file = Path("/content/drive/MyDrive/finalyearproject/notes/notes6.pkl")
    with open(notes_file, "rb") as file:
        notes = pickle.load(file)

    # Extract individual components for each note
    all_pitches, all_durations, all_offsets = [], [], []
    for item in notes:
        parts = item.split(":")
        all_pitches.append(parts[0])
        all_durations.append(parts[1])
        all_offsets.append(parts[2])

    # Unique vocabularies
    pitchnames = sorted(set(all_pitches))
    durations = sorted(set(all_durations))
    offsets = sorted(set(all_offsets))

    for seq_length, dropout_rate in product(sequence_lengths, dropout_rates):
        print(f"\nTraining with Sequence Length: {seq_length}, Dropout Rate: {dropout_rate}")

        # Prepare sequences
        n_vocab_offsets = len(offsets)
        network_input_offsets, network_output_offsets = prepare_sequences(all_offsets, n_vocab_offsets, seq_length)

        n_vocab_notes = len(pitchnames)
        network_input_notes, network_output_notes = prepare_sequences(all_pitches, n_vocab_notes, seq_length)

        n_vocab_durations = len(durations)
        network_input_durations, network_output_durations = prepare_sequences(all_durations, n_vocab_durations, seq_length)

        # Create model for current configuration
        model = create_network(
            network_input_notes, n_vocab_notes,
            network_input_durations, n_vocab_durations,
            network_input_offsets, n_vocab_offsets,
            dropout_rate
        )

        # Train model with validation tracking
        history = train(
            model,
            network_input_notes, network_input_durations, network_input_offsets,
            network_output_notes, network_output_durations, network_output_offsets,
            seq_length, dropout_rate,
            validation_split=0.2
        )

        # Store both training and validation loss curves
        results.append({
            'sequence_length': seq_length,
            'dropout_rate': dropout_rate,
            'final_loss': history.history['loss'][-1],
            'final_train_loss': ','.join(map(str, history.history['loss'])),
            'final_val_loss': ','.join(map(str, history.history['val_loss']))
        })

        # Save model for this configuration
        model_save_path = Path(f"/content/drive/MyDrive/finalyearproject/gridsearch/models/model_seq{seq_length}_drop{dropout_rate}.keras")
        model.save(model_save_path)

        # Save results to CSV after each iteration
        results_df = pd.DataFrame(results)
        results_df.to_csv('/content/drive/MyDrive/finalyearproject/gridsearch/plots/grid_search_resultsnew2.csv', index=False)

    print("\nGrid Search Results:")
    for result in results:
        print(f"Sequence Length: {result['sequence_length']}, Dropout Rate: {result['dropout_rate']}, Final Loss: {result['final_loss']}")

    # Create loss curve plots: one set for training, one for validation
    results_df = pd.DataFrame(results)
    plot_loss_curves(results_df)      # training loss graphs (3 plots)
    plot_val_loss_curves(results_df)   # validation loss graphs (3 plots)

    return results

In [ ]:
# Run the grid search
grid_search_training()


Training with Sequence Length: 64, Dropout Rate: 0.1
Epoch 1/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 14s 37ms/step - Duration_loss: 2.7142 - Note_loss: 5.1890 - Offset_loss: 3.1876 - loss: 11.0909 - val_Duration_loss: 1.8962 - val_Note_loss: 4.8483 - val_Offset_loss: 1.9012 - val_loss: 8.6410
Epoch 2/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - Duration_loss: 1.7091 - Note_loss: 4.1409 - Offset_loss: 1.8228 - loss: 7.6727 - val_Duration_loss: 1.8055 - val_Note_loss: 5.1180 - val_Offset_loss: 1.6875 - val_loss: 8.6175
Epoch 3/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - Duration_loss: 1.5171 - Note_loss: 3.9102 - Offset_loss: 1.4600 - loss: 6.8872 - val_Duration_loss: 1.5101 - val_Note_loss: 4.0902 - val_Offset_loss: 1.3927 - val_loss: 7.0151
Epoch 4/50
173/173 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - Duration_loss: 1.4213 - Note_loss: 3.7223 - Offset_loss: 1.3247 - loss: 6.4682 - val_Duration_loss: 1.4651 - val_Note_loss: 4.0089 - val_Offset_loss: 1.3947 - val_loss: 6.9009
Epoch 5/50
173/173 ━

[{'sequence_length': 64,
  'dropout_rate': 0.1,
  'final_loss': 0.375548779964447,
  'final_train_loss': '9.710179328918457,7.362041473388672,6.734018802642822,6.4437737464904785,6.234638214111328,5.9785542488098145,5.744400501251221,5.559484481811523,5.334316253662109,5.1481242179870605,4.970940589904785,4.758142471313477,4.591461181640625,4.306638717651367,4.134425640106201,3.921826124191284,3.6800310611724854,3.477922201156616,3.289721727371216,3.09765362739563,2.892505645751953,2.704911947250366,2.5175018310546875,2.320296287536621,2.1731722354888916,2.0196661949157715,1.877205729484558,1.9412564039230347,1.692794919013977,1.4660075902938843,1.336429476737976,1.2297827005386353,1.09054434299469,1.055299162864685,0.9761243462562561,0.8867116570472717,0.8181325793266296,0.7413403987884521,0.7047544717788696,0.6696919798851013,0.6728218197822571,0.6006828546524048,0.6260506510734558,0.8585234880447388,0.545764148235321,0.4937392771244049,0.4223874509334564,0.4314926266670227,0.3778898

In [ ]:
def plot_loss_curves(results_df):
    """
    Create and save training loss curve plots for different sequence lengths and dropout rates.
    Saves one PNG per sequence length.
    """
    sequence_lengths = [64, 128, 192]
    dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5]
    colors = ['blue', 'green', 'red', 'purple', 'orange']

    for seq_length in sequence_lengths:
        plt.figure(figsize=(10, 6))
        seq_results = results_df[results_df['sequence_length'] == seq_length]
        for j, dropout_rate in enumerate(dropout_rates):
            result = seq_results[seq_results['dropout_rate'] == dropout_rate].iloc[0]
            losses = list(map(float, result['final_train_loss'].split(',')))
            plt.plot(range(1, len(losses)+1), losses,
                     label=f'Dropout {dropout_rate}',
                     color=colors[j])
        plt.title(f'Training Loss Curves - Sequence Length {seq_length}')
        plt.xlabel('Epoch')
        plt.ylabel('Training Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'/content/drive/MyDrive/finalyearproject/gridsearch/plots/loss_curves_seq{seq_length}2.png')
        plt.close()

In [ ]:
def plot_val_loss_curves(results_df):
    """
    Create and save validation loss curve plots for different sequence lengths and dropout rates.
    Saves one PNG per sequence length.
    """
    sequence_lengths = [64, 128, 192]
    dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5]
    colors = ['blue', 'green', 'red', 'purple', 'orange']

    for seq_length in sequence_lengths:
        plt.figure(figsize=(10, 6))
        seq_results = results_df[results_df['sequence_length'] == seq_length]
        for j, dropout_rate in enumerate(dropout_rates):
            result = seq_results[seq_results['dropout_rate'] == dropout_rate].iloc[0]
            losses = list(map(float, result['final_val_loss'].split(',')))
            plt.plot(range(1, len(losses)+1), losses,
                     label=f'Dropout {dropout_rate}',
                     color=colors[j])
        plt.title(f'Validation Loss Curves - Sequence Length {seq_length}')
        plt.xlabel('Epoch')
        plt.ylabel('Validation Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f'/content/drive/MyDrive/finalyearproject/gridsearch/plots/val_loss_curves_seq{seq_length}2.png')
        plt.close()